<a href="https://colab.research.google.com/github/beyg1/Q3/blob/main/FunctionCall_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U langchain-google-genai # google's generative ai installation with langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 868.6 kB/s eta 0:00:00


In [2]:
from google.colab import userdata #importing gemini key
from langchain_google_genai import ChatGoogleGenerativeAI
#connecting LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",api_key=userdata.get('GOOGLE_API_KEY'))
llm.invoke("sup")

AIMessage(content="Sup yourself!  How's it going?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-63e44add-d80e-4ae7-ba23-fe0d2660ead8-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [3]:
#Tool/Function Calling
from langchain_core.tools import tool

@tool
def func1(i:int):
  """Return Function1"""
  return i+50/10
tools = [func1]

In [4]:
#A tool func1 hasbeen created but hasnt been passed to the llm. that's why no answer
llm.invoke("What is Function1 of 1")

AIMessage(content='"Function1 of 1" isn\'t a standard mathematical or programming term.  It\'s likely referring to a specific context, such as:\n\n* **A numbered function in a list or sequence:**  There\'s only one function, and it\'s the first (and only) one.  This is common in documentation or code where multiple functions are described.\n\n* **Progress indicator:**  It might be part of a progress bar or status update indicating that one out of one function has completed.\n\n* **Part of a filename or identifier:**  It\'s possible this is a portion of a longer name, like `Function1_of_1.py` which suggests a Python file.\n\nWithout more information about where you encountered this phrase, it\'s impossible to give a more precise answer.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-ee86ef7f-cdbf-42f9-a9a0-4b7382e5f0de-0', usage_metadata={'input_tokens': 8, 'output_tokens': 

In [5]:
llm_tools = llm.bind_tools(tools) ## Bind tools to the LLM

In [6]:
llm_tools.invoke("What is func1 for 1") #llm now recognizez the tool/func called but still no content

AIMessage(content='', additional_kwargs={'function_call': {'name': 'func1', 'arguments': '{"i": 1.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-54058c91-bf8b-440e-9420-4198080d06c8-0', tool_calls=[{'name': 'func1', 'args': {'i': 1.0}, 'id': '841cf0b2-d48f-43b4-bb8d-7e87dfab9e27', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 3, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}})

In [7]:
from IPython.display import display
#The display function is used to nicely present outputs within a Jupyter Notebook environment.

ai_msg = llm_tools.invoke("What is func1 for 1")
display(ai_msg)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'func1', 'arguments': '{"i": 1.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9822fb2f-6bd8-4355-a7d2-60dee1bbea7a-0', tool_calls=[{'name': 'func1', 'args': {'i': 1.0}, 'id': 'a8c44a39-841a-43e8-b5da-ae27dec35cd9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 3, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}})

In [9]:
# create message(Prompts & Responses) history
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
query = "What is func1 for 1"
messages = [HumanMessage(query)]
display(messages)
#4 kinds of messages: Human message, System message, AI message, Tool message

[HumanMessage(content='What is func1 for 1', additional_kwargs={}, response_metadata={})]

In [10]:
ai_msg_tools = llm_tools.invoke(messages)
messages.append(ai_msg_tools)
display(messages)
# Remember messages is our list of messages in the conversation? This line adds
# the LLM's response (ai_msg_tools) to that list. So now messages has both our
# original query and the LLM's answer. This is useful for keeping track of the conversation history.

[HumanMessage(content='What is func1 for 1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'func1', 'arguments': '{"i": 1.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-28d4d41a-2deb-4122-877d-1bae785a8665-0', tool_calls=[{'name': 'func1', 'args': {'i': 1.0}, 'id': '189a3db3-dc94-4a1c-8080-0fd2b5826d32', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 3, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}})]

In [11]:
ai_msg_tools.tool_calls

[{'name': 'func1',
  'args': {'i': 1.0},
  'id': '189a3db3-dc94-4a1c-8080-0fd2b5826d32',
  'type': 'tool_call'}]

In [12]:
#Finally to get AI message, in this case tool message, which will be passed to llm:
for tool_call in ai_msg_tools.tool_calls:
    selected_tool = {
        "func1": func1,
       #add more tools
      }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    display(tool_msg)
    messages.append(tool_msg)

ToolMessage(content='6.0', name='func1', tool_call_id='189a3db3-dc94-4a1c-8080-0fd2b5826d32')

In [13]:
display(messages)
#prompt (human message) and tool message is now saved in messages

[HumanMessage(content='What is func1 for 1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'func1', 'arguments': '{"i": 1.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-28d4d41a-2deb-4122-877d-1bae785a8665-0', tool_calls=[{'name': 'func1', 'args': {'i': 1.0}, 'id': '189a3db3-dc94-4a1c-8080-0fd2b5826d32', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 3, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='6.0', name='func1', tool_call_id='189a3db3-dc94-4a1c-8080-0fd2b5826d32')]

In [16]:
# Invoke the llm with messages
response = llm_tools.invoke(messages)
messages.append(response)

In [21]:
#for a nice display format
from IPython.display import Markdown

Markdown(response.content)

func1 for 1 returns `{"func1_response": "{"output": 6}"}`.

In [18]:
display(messages)

[HumanMessage(content='What is func1 for 1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'func1', 'arguments': '{"i": 1.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-28d4d41a-2deb-4122-877d-1bae785a8665-0', tool_calls=[{'name': 'func1', 'args': {'i': 1.0}, 'id': '189a3db3-dc94-4a1c-8080-0fd2b5826d32', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 3, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='6.0', name='func1', tool_call_id='189a3db3-dc94-4a1c-8080-0fd2b5826d32'),
 AIMessage(content='func1 for 1 returns `{"func1_response": "{"output": 6}"}`.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-ff781a06-536f-4859-ac57-e8356c80c25f-0', usage_meta